In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import cv2
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Concatenate,Input
from tensorflow.keras.layers import (Dense, Dropout, Activation,
                                     Flatten, GlobalAveragePooling2D,GlobalMaxPooling2D )
from tensorflow.keras.models import Sequential, Model, load_model
import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options=tf.compat.v1.GPUOptions(allow_growth=True))
sess = tf.compat.v1.Session(config=config)

##  資料讀入及前處理

In [54]:
train_path =  "D:/score_0623/"

x_data_train = []
y_data_train = []
for roots, dirs, files in os.walk(train_path):
    for each in files:
        if each.find('checkpoint') == -1:
            x_data_train.append(os.path.join(roots.split("/")[-1], each))
            y_data_train.append(roots.split("/")[-1])

In [4]:
data_train = pd.DataFrame({})
data_train['img_path'] = x_data_train
data_train['label'] = y_data_train

In [38]:
data_train

,img_path,label
0,0\img8_0.png,0
1,0\img8_1.png,0
2,0\img8_10.png,0
3,0\img8_100.png,0
4,0\img8_1000.png,0
...,...,...
40782,9\img7_995.png,9
40783,9\img7_996.png,9
40784,9\img7_997.png,9
40785,9\img7_998.png,9


In [5]:
print(len(x_data_train),len(y_data_train))

40787 40787


# 讀入測試資料

In [6]:
# test_path = "c:/word/" #讀入測試資料

# x_data_test = []
# y_data_test = []
# for roots, dirs, files in os.walk(test_path):
#     for each in files:
#         if each.find('checkpoint') == -1:
#             x_data_test.append(os.path.join(roots.split("/")[-1], each))
#             y_data_test.append(roots.split("/")[-1])

In [7]:
# data_test = pd.DataFrame({})
# data_test['img_path'] = x_data_test
# data_test['label'] = y_data_test
# data_test['label'] = data_test['img_path'].str.replace('.png', '')

In [8]:
# print(len(x_data_test),len(y_data_test))

In [9]:
# #Label導入
# class_map = pd.read_csv('d:/python/class_map_0611.csv', index_col=0) 
# class_map = class_map.to_dict()['class']

In [10]:
#class_map

In [11]:
# data=data_train.copy()
# ax = data.groupby('label').count().plot(kind='bar', figsize=(10, 5), fontsize=20, legend=False)
# grouped_values = data.groupby('label').count()['img_path'].values
# for idx, value in enumerate(grouped_values):
#     ax.text(idx, value-value//8, value, color="black", ha="center", fontsize=25)
# ax.set_title("TrainData", fontsize=30)
# ax.set_xlabel("", fontsize=14)
# ax.set_ylabel("", fontsize=14)

In [12]:
from sklearn.model_selection import train_test_split

train_list, test_list = train_test_split(data_train,
                                         test_size=0.1,
                                         random_state=35,
                                         stratify=data_train['label'].values)

In [13]:
img_size = 112
num_class = len(data_train['label'].unique())
num_class

10

## 讀入 Pre-trained Model

In [14]:
from tensorflow.keras.applications.xception import (Xception, preprocess_input)

In [15]:
input_tensor = Input(shape = (112,112, 3))
model = Xception(include_top=False, weights='imagenet', input_tensor=input_tensor)
x_max = GlobalMaxPooling2D()(model.output)
x_avg = GlobalAveragePooling2D()(model.output)
x_2 = (model.output)*(-1)
x_min_1 = GlobalMaxPooling2D()(x_2)
x_min = x_min_1*(-1)
x = Concatenate(axis=-1)([x_max,x_avg,x_min])
output = Dense(1,activation = 'relu')(x)
model1 = Model(input_tensor, output)

In [16]:
model1.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 112, 112, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 55, 55, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 55, 55, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 55, 55, 32)   0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

# 決定 Pre-Train 參數量

In [17]:
model1.trainable = True

# trainable_layer = 10
# for layer in pre_model.layers[:-trainable_layer]:
#     layer.trainable = False

# for layer in model.layers:
#     print(layer, layer.trainable)

# model.summary()

In [18]:
learning_rate = 1e-5
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

model1.compile(optimizer=optimizer,loss = 'mse',metrics=["mse","mae","mape"])

# Data Augmentation

In [19]:
batch_size = 64
num_steps = len(train_list) // batch_size + 1
num_epochs = 30

In [20]:
train_datagen = ImageDataGenerator(
                                   rotation_range=10,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=False,
                                   zoom_range = [1.2,1.2],
                                   fill_mode='nearest',
                                   preprocessing_function=preprocess_input
                                  )
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
data_test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [21]:
img_shape = (img_size, img_size)
# train data 讀取
train_generator = train_datagen.flow_from_dataframe(
                                                    dataframe=train_list,
                                                    directory=train_path,
                                                    x_col="img_path",
                                                    y_col="label",
                                                    target_size=img_shape,
                                                    batch_size=batch_size,
                                                    class_mode='sparse')

# valid_generator = valid_datagen.flow_from_dataframe(
#                                                     dataframe=val_list,
#                                                     directory=None,
#                                                     x_col="img_path",
#                                                     y_col="ll",
#                                                     target_size=img_shape,
#                                                     batch_size=batch_size,
#                                                     class_mode='categorical',
#                                                     shuffle=False)


test_generator = test_datagen.flow_from_dataframe(
                                                    dataframe=test_list,
                                                    directory=train_path,
                                                    x_col="img_path",
                                                    y_col="label",
                                                    target_size=img_shape,
                                                    batch_size=batch_size,
                                                    class_mode='sparse',
                                                    shuffle=False)
# 用在不知道 label 的資料讀取上(自己手寫資料)
# data_test_generator = data_test_datagen.flow_from_dataframe(
#                                                     dataframe=data_test,
#                                                     directory=test_path,
#                                                     x_col="img_path",
#                                                     target_size=img_shape,
#                                                     batch_size=batch_size,
#                                                     class_mode=None,
#                                                     shuffle=False)


Found 36708 validated image filenames belonging to 10 classes.
Found 4079 validated image filenames belonging to 10 classes.


In [22]:
train_list

,img_path,label
4734,1\img9_1579.png,1
6134,1\img9_2841.png,1
37762,9\img7_1959.png,9
32869,8\img6_1234.png,8
27351,6\img4_3635.png,6
...,...,...
2441,0\img8_32.png,0
7098,1\img9_3711.png,1
7227,1\img9_3828.png,1
33462,8\img6_177.png,8


In [23]:
test_list

,img_path,label
4350,1\img9_1232.png,1
2302,0\img8_3074.png,0
13091,3\img1_1767.png,3
9356,2\img0_2072.png,2
12548,3\img1_1277.png,3
...,...,...
4718,1\img9_1564.png,1
17196,4\img2_1786.png,4
15979,3\img1_684.png,3
3325,0\img8_3998.png,0


In [24]:
for i in train_generator:
    print(i[1])
    break

[4. 8. 6. 8. 4. 9. 6. 1. 5. 0. 1. 2. 2. 7. 9. 9. 2. 7. 8. 3. 6. 1. 7. 1.
 5. 7. 0. 0. 9. 9. 7. 1. 0. 0. 8. 6. 4. 0. 1. 8. 1. 0. 5. 9. 3. 9. 4. 5.
 6. 7. 5. 5. 9. 8. 1. 0. 8. 6. 8. 2. 7. 6. 7. 0.]


In [25]:
for i in test_generator:
    print(i[1])
    break

[1. 0. 3. 2. 3. 2. 2. 6. 5. 9. 1. 4. 0. 6. 1. 9. 8. 2. 2. 3. 4. 4. 9. 7.
 8. 9. 1. 7. 4. 4. 8. 6. 7. 8. 5. 9. 0. 9. 7. 0. 6. 3. 5. 2. 9. 6. 8. 7.
 1. 3. 4. 4. 7. 1. 8. 9. 4. 4. 5. 5. 7. 2. 8. 9.]


## 開始訓練模型

In [25]:
model_dir = 'model-logs/xception_0625'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

logfiles = model_dir + '/{}-{}'.format('basic_model',
                                       model.__class__.__name__)
model_cbk = keras.callbacks.TensorBoard(log_dir=logfiles,
                                        histogram_freq=1)

modelfiles = model_dir + '/{}-best-model.h5'.format('basic_model')
model_mckp = keras.callbacks.ModelCheckpoint(modelfiles,
                                             monitor='val_loss',
                                             save_best_only=True)

earlystop = keras.callbacks.EarlyStopping(monitor='val_loss',
                                          patience=5,
                                          verbose=1)


callbacks_list = [model_cbk, model_mckp, earlystop]

In [26]:
history = model1.fit_generator(train_generator,
                              steps_per_epoch=num_steps,
                              epochs=num_epochs,
                              #validation_data=test_generator,
                              callbacks=callbacks_list)

C:\ProgramData\Anaconda3\envs\tf_night\lib\site-packages\tensorflow\python\keras\engine\training.py:1950: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
C:\ProgramData\Anaconda3\envs\tf_night\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 1/30
574/574 [==============================] - 192s 310ms/step - loss: 3.5473 - mse: 3.5473 - mae: 1.2747 - mape: 4270283.5000
Epoch 2/30
574/574 [==============================] - 122s 212ms/step - loss: 1.0668 - mse: 1.0668 - mae: 0.7784 - mape: 3559797.2500 - loss: 1.0735 - mse: 1.0735 - mae: 0.781 - ETA: 1s - loss: 1.0686 - mse: 1.0686 - mae: 0.7791 - mape: 
Epoch 3/30
574/574 [==============================] - 122s 212ms/step - loss: 0.8395 - mse: 0.8395 - mae: 0.6766 - mape: 1719976.8750
Epoch 4/30
574/574 [==============================] - 122s 212ms/step - loss: 0.7286 - mse: 0.7286 - mae: 0.6199 - mape: 905200.8750: 15s - loss: 0.7310 - mse: 0.7310 - mae
Epoch 5/30
574/574 [==============================] - 125s 218ms/step - loss: 0.6443 - mse: 0.6443 - mae: 0.5741 - mape: 605101.6250
Epoch 6/30
574/574 [==============================] - 123s 215ms/step - loss: 0.5637 - mse: 0.5637 - mae: 0.5300 - mape: 947340.0625
Epoch 7/30
574/574 [==============================] - 1

In [27]:
model1.save('best_model_0625_112.h5')

## 訓練結果視覺化

In [26]:
model1 = load_model('d:/python/model-logs/xception_0625/best_model_0625_112.h5')



In [27]:
test_list.reset_index(drop=True, inplace=True)

In [28]:
test_list

,img_path,label
0,1\img9_1232.png,1
1,0\img8_3074.png,0
2,3\img1_1767.png,3
3,2\img0_2072.png,2
4,3\img1_1277.png,3
...,...,...
4074,1\img9_1564.png,1
4075,4\img2_1786.png,4
4076,3\img1_684.png,3
4077,0\img8_3998.png,0


In [71]:
x,y=[],[]
for i in range(test_list.shape[0]):
    name = str(test_list.loc[i,'img_path'])
    name2 = "D:/score_0623/"+name[0:1]+'/' +name[2::]
    img=cv2.imread(name2)

    img=cv2.resize(img,(112,112))
    img=img.astype("float32")     
    x.append(img)
    y.append(test_list.loc[i,"label"])
    
x_val=np.array(x,dtype="float32")  
y_val=np.array(y,dtype="float32")
x_val = preprocess_input(x_val)

In [74]:
#ML的評量
from sklearn.metrics import mean_squared_error     #均方誤差
from sklearn.metrics import mean_absolute_error    #平方絕對誤差
from sklearn.metrics import r2_score

def score(y_valid,P):
    a=mean_squared_error(y_valid,P)
    b=mean_absolute_error(y_valid,P)
    c=r2_score(y_valid,P)
    print(f'均方誤差:{a},平方絕對誤差:{b},r2:{c}')

P=model1.predict(x_val)
kk=y_val
score(kk,P)

均方誤差:8.226165771484375,平方絕對誤差:2.3003478050231934,r2:0.004958628477266802
